# Use pgvector for Semantic Search in Postgresql
Before running this script, run 00_generate_embeddings.ipynb to generate the embeddings. Also start the postgresql database using docker-compose

In [1]:
# Load libraries
import pandas as pd
import psycopg2
from pgvector.psycopg2 import register_vector
from sentence_transformers import SentenceTransformer

MODEL = 'paraphrase-multilingual-MiniLM-L12-v2' # https://www.sbert.net/docs/pretrained_models.html
TRANSFORMED_DATA_PATH = 'data/transformed'

model = SentenceTransformer(MODEL)

file_name = 'dataset_1.parquet'

In [2]:
# Set up a connection to the local db
conn = psycopg2.connect(
        dbname='vectordb',
        user='testuser',
        password='testpwd',
        host='localhost'
)
conn.autocommit = True  

In [3]:
# Read data to be inserted into the db
df = pd.read_parquet(f'{TRANSFORMED_DATA_PATH}/{file_name}')

# Write insert statement
insert_query = '''
    INSERT INTO
        chunks
            (pdf, page, block_id, block_text, chunk_id, chunk_text, embedding)
        VALUES
            (%s, %s, %s, %s, %s, %s, %s)
    '''

# Upload data into db
with conn.cursor() as cur:
    register_vector(cur)
    cur.executemany(insert_query, df.values.tolist())
    conn.commit()


In [4]:
# Set a search term
search_term = 'Ethik'

The following query uses cosine similarity for search (https://github.com/pgvector/pgvector). However, for a usable product that can also handle full text search, combine the existing query with a full text search (https://www.postgresql.org/docs/current/textsearch.html). Also keep in mind that for a bigger corpus it is advised to use an index to speed up the search.

parldebatescanner.org uses the cosine similarity score and a ts_rank to blend the results of a semantic and full text search. It uses an IVFFlat index on the transformer embeddings and a GIN index on the ts vector to speed up retrieval.

In [5]:
# Generate embedding for search term
search_embedding = model.encode(search_term, show_progress_bar=False)

# Run cosine similarity search using pgvectgor in postgresql
with conn.cursor() as cur:
    register_vector(cur)
    query = '''
    SELECT
        pdf, page, block_id, block_text, chunk_id, chunk_text, embedding <=> %(embedding)s AS score
     FROM
        chunks
    ORDER BY
        embedding <=> %(embedding)s ASC
    LIMIT
        20
    ''' 
    cur.execute(query, {'embedding': search_embedding})
    df_result = pd.DataFrame(cur.fetchall(), columns=[['pdf', 'page', 'block_id', 'block_text', 'chunk_id', 'chunk_text', 'score']])

# Print the results
df_result

,pdf,page,block_id,block_text,chunk_id,chunk_text,score
0,fedlex-data-admin-ch-eli-dl-proj-2023-5-cons_1...,1214,3656,So wie das Recht auf Wissen und Nicht-Wissen h...,0,So wie das Recht auf Wissen und Nicht-Wissen h...,0.379465
1,fedlex-data-admin-ch-eli-dl-proj-2023-5-cons_1...,1232,3721,So wie das Recht auf Wissen und Nicht-Wissen h...,0,So wie das Recht auf Wissen und Nicht-Wissen h...,0.379465
2,fedlex-data-admin-ch-eli-dl-proj-2023-5-cons_1...,1222,3681,So wie das Recht auf Wissen und Nicht-Wissen h...,0,So wie das Recht auf Wissen und Nicht-Wissen h...,0.379465
3,fedlex-data-admin-ch-eli-dl-proj-2023-5-cons_1...,539,1242,communiquer. Il convient de tenir compte de l’...,1,ertes fortuites alors que les règles clinique...,0.381037
4,fedlex-data-admin-ch-eli-dl-proj-2023-5-cons_1...,2514,7585,SSPTC 1d L'art. 1 de l'ordonnance sur l'orga...,0,SSPTC 1d L'art. 1 de l'ordonnance sur l'orga...,0.383583
5,fedlex-data-admin-ch-eli-dl-proj-2023-5-cons_1...,24,53,Ein ethisch nicht zu lösendes Dilemma bei Zufa...,0,Ein ethisch nicht zu lösendes Dilemma bei Zufa...,0.407778
6,fedlex-data-admin-ch-eli-dl-proj-2023-5-cons_1...,2903,8344,1 1 i 1 La commission d’éthique de l...,0,1 1 i 1 La commission d’éthique de l...,0.408061
7,fedlex-data-admin-ch-eli-dl-proj-2023-5-cons_1...,2716,7822,Ein ethisch nicht zu lösendes Dilemma bei Zufa...,0,Ein ethisch nicht zu lösendes Dilemma bei Zufa...,0.411166
8,fedlex-data-admin-ch-eli-dl-proj-2023-5-cons_1...,2736,7913,swissethics Expertise der Mitglieder der Ethi...,0,swissethics Expertise der Mitglieder der Ethi...,0.420604
9,fedlex-data-admin-ch-eli-dl-proj-2023-5-cons_1...,44,143,swissethics Expertise der Mitglieder der Ethi...,0,swissethics Expertise der Mitglieder der Ethi...,0.420604


In [6]:
conn.close()